In [162]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.graph_objs as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output,Input
from io import BytesIO
from wordcloud import WordCloud
import base64

In [7]:
df = pd.read_csv("globalterrorismdb_0718dist.csv",  encoding='ISO-8859–1')

C:\Users\Agam\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning:

Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types.Specify dtype option on import or set low_memory=False.



In [163]:
app = dash.Dash()

In [164]:
a  = df['country_txt'].unique()
a.sort()
countries = []
for country in a:
    countries.append({'label':country,'value':country})
countries.append({'label':'World','value':'World'})    

In [43]:
    labels = []
    parents = []
    values = []
    
    for region in df['region_txt'].unique():
        labels.append(region)
        parents.append("")
        values.append(df[df['region_txt'] == region]['iyear'].count())
        
    for country in df['country_txt'].unique():
        labels.append(country)
        parents.append(df[df['country_txt'] == country]['region_txt'].unique()[0])
        values.append(df[df['country_txt'] == country]['iyear'].count())
        
    trace_sun =[go.Sunburst(
    labels=labels,
    parents=parents,
    values=values,
    branchvalues="total",
    )]
    layout_sun = go.Layout(title='sun plot')
    
    fig = go.Figure(data=trace_sun,layout=layout_sun)
    pyo.plot(fig,filename='sun.html')

'sun.html'

In [165]:
app.layout = html.Div(children = [
                   
                html.P(dcc.Dropdown(id = 'country_picker',options = countries,value = 'World')),
                html.P(dcc.Graph(id='graph0')),
                html.P(html.Div(children = [ dcc.Graph(id='graph1',style={'display': 'inline-block'}),
                                         dcc.Graph(id='graph2',style={'display': 'inline-block'})])),
                html.P(dcc.Slider(id = 'year_picker',min=df['iyear'].min(),max=df['iyear'].max(),step =2,value = df['iyear'].min(),
                              marks = {i : i for i in range(df['iyear'].min(),df['iyear'].max(),2)})),
                html.P(dcc.Dropdown(id = 'wordcloud',options=countries,value = 'India',style={"padding": 10})),
                html.P(html.Img(id="image_wc",className='container', style={'width': '96%','padding-left':'3', 'padding-right':'1%'}))
                
                   
])

In [166]:
@app.callback(Output('graph0','figure'),[Input('country_picker','value')] )
def country_selecter(selected_country):
    filtered_df = df
    if(selected_country!='World'):
        filtered_df = df[df['country_txt'] == selected_country]
    
    x_data = []
    y_data = []
    
    for year in filtered_df['iyear'].unique():
        x_data.append(year)
        y_data.append(filtered_df[(filtered_df['iyear'] == year)]['iyear'].count())
    
    trace = [go.Bar(x = x_data,y = y_data)]
    
    layout = go.Layout(title = 'region',xaxis = {'title' : 'year'},yaxis = {'title':'number of attacks'})
    
    return {'data' : trace, 'layout' : layout}
        

In [172]:
@app.callback(Output('graph1','figure'),[Input('year_picker','value')] )
def year_selecter(selected_year):
    
    filtered_df = df[df['iyear'] == selected_year]
    trace = [] 
    df['location'] = df['country_txt'] + " , " + df['city']
    for region in filtered_df['region_txt'].unique():
        trace.append(go.Scattergeo(text = filtered_df[filtered_df['region_txt'] == region]['location'],
                               lon=filtered_df[filtered_df['region_txt'] == region]['longitude'],
                               lat=filtered_df[filtered_df['region_txt'] == region]['latitude'],
                           mode = 'markers',marker=dict(size = 3),
                                  name = region))
    
    layout = go.Layout(title = 'World Map',xaxis = {'title' : 'region'},yaxis = {'title':'no of attacks'})
    
    return {'data' : trace, 'layout' : layout}

DuplicateCallbackOutput: 
You have already assigned a callback to the output
with ID "graph1" and property "figure". An output can only have
a single callback function. Try combining your inputs and
callback functions together into one function.


In [168]:
@app.callback(Output('graph2','figure'),[Input('year_picker','value')] )
def year_selecter(selected_year):
    
    filtered_df = df[df['iyear'] == selected_year]
    
    x_data = []
    y_data = []
    trace = []
    for region in filtered_df['region_txt'].unique():
        x_data.append(region)
        y_data.append(filtered_df[(filtered_df['region_txt'] == region)]['iyear'].count())
        y_single = filtered_df[(filtered_df['region_txt'] == region)]['iyear'].count()
        trace.append(go.Bar(x=[region],y = [y_single],name = region))
    
    layout = go.Layout(title = 'region',xaxis = {'title' : 'region'},yaxis = {'title':'no of attacks'})
    
    return {'data' : trace, 'layout' : layout}
        
    
    
    
    
    

# WordCloud

In [169]:
def plot_wordcloud1(df,b):
    d = df[df['country_txt'] == 'India']['attacktype1_txt'].astype(str).values.tolist()
    d = ' '.join(d)
    t = df[df['country_txt'] == 'India']['targsubtype1_txt'].astype(str).values.tolist()
    t = ' '.join(t)
    d = d + t
    t = df[df['country_txt'] == 'India']['gname'].astype(str).values.tolist()
    t = ' '.join(t)
    d = d + t
    wc = WordCloud(background_color='white', width=1000, height=500)
    wc.generate(d)
    return wc.to_image()


In [170]:
@app.callback(Output('image_wc', 'src'), [Input('wordcloud', 'value')])
def make_image(b):
    img = BytesIO()
    plot_wordcloud1(df,b).save(img, format='PNG')
    return 'data:image/png;base64,{}'.format(base64.b64encode(img.getvalue()).decode())

In [171]:
if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Mar/2021 17:51:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2021 17:51:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2021 17:51:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2021 17:51:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2021 17:51:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2021 17:51:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2021 17:51:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2021 17:51:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2021 17:51:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2021 17:51:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2021 17:51:45] "POST /_dash-update-component HTTP/1.1" 200 -
